In order to run this, you should have vector database `(vector_db_)` already stored as .zip. You can download it using the link
https://drive.google.com/drive/folders/1o2Y-BLNWNFT5zesysd2n3QZPtN0Jro3I?usp=drive_link

In [1]:
# Install the required libraries
# Note: After running this code, the kernel needs to be restarted.

!pip install --quiet chromadb langchain-chroma langchain langchain-community langchain-text-splitters langchain-groq unstructured



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Run the code snippet below it has every library included.

In [2]:
import os
import pandas as pd
from transformers import pipeline

# Langchain modules
from langchain.document_loaders import UnstructuredFileLoader, PyPDFDirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TokenTextSplitter,
)
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"


# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2')
# embeddings = HuggingFaceEmbeddings() # Make sure you use the same embedders as you used to embed the original dataset into vector database.


<ipython-input-2-7fa1c931af47>:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

There are two ways to read and predict in the model, Manual and Automatic

# Automatic entry of vector database


## Unziping

In [3]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")

Extracting /content/parts_data_vectorDB.zip...
Extraction completed for parts_data_vectorDB.zip!
Extracting /content/service_data_vectorDB.zip...
Extraction completed for service_data_vectorDB.zip!


## Reading the vector database

In [6]:
import os
from langchain.vectorstores import Chroma

# Directory containing vector databases
vector_db_dir = "/content/content/"
embedding_function = embeddings  # Ensure this is defined earlier in your code

# Initialize a list to store retrievers
retrievers = []

# Iterate through all subdirectories in the specified directory
for sub_dir in os.listdir(vector_db_dir):
    full_path = os.path.join(vector_db_dir, sub_dir)
    if os.path.isdir(full_path):  # Check if it's a directory
        print(f"Loading vector database from: {full_path}")

        # Load the vector database
        vectordb = Chroma(
            persist_directory=full_path,
            embedding_function=embedding_function
        )

        # Create a retriever and add it to the list
        retrievers.append(vectordb.as_retriever())

# Define a combined retriever function
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        # Retrieve documents from each retriever
        combined_results.extend(retriever.get_relevant_documents(query))

    return combined_results

# Example Usage
query = "What is the price of a fuel injector for 2005 toyota rav 4 ?"
results = combined_retriever(query)
print(f"Retrieved {len(results)} documents.")

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")

Loading vector database from: /content/content/service_data
Loading vector database from: /content/content/parts_data
Retrieved 8 documents.


## Chatbot finetuning and chatting

In [7]:
# Create a prompt template
chatbot_prompt = """
You are an automotive parts assistant. When a user asks about their vehicle, you will refer to relevant documents and provide guidance in a concise, clear manner. Your goal is to stay under 1,000 tokens for each response, including all necessary details.

Here are the steps to follow:

1. **Identify the Vehicle and Parts**: Determine what part the user is asking about based on their question. If it’s unclear, ask clarifying questions to understand the model, make, and year of the vehicle.

2. **Provide Pricing**: Always provide the price for the part requested. If you don’t have the exact year of the vehicle the user asks about, provide the price for the earliest year available in your database and inform the user that you cannot help with the exact year requested. You should always mention the earliest year available, even if the part is not available for the requested year.

3. **Include Related Subcategories**: If the part requested falls under a category that has subcategories (e.g., "engine parts" has "fuel injectors", "oil filters", etc.), list those subcategories with their prices, if available.

4. **Mention Fluids**: If the part requested is related to fluids (e.g., oil, transmission fluid), also mention the fluids associated with the part and their availability/price.

5. **Maintain Chat Context**: Keep track of previous conversations and refer to them when necessary to provide consistent follow-up answers. For example, if the user has already asked about a part and later asks about another part from the same vehicle, refer back to previous details such as model, year, or part-related info.

6. **Structure the Response**:
   - Begin by acknowledging the vehicle type and confirming details, especially if the model year or make was mentioned.
   - Provide the price for the part.
   - List any relevant subcategories and related fluids.
   - If no exact match for the year is found, state the earliest year available in the database and explain the limitation.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.invoke(
            {"input_documents": combined_results, "question": query}, return_only_outputs=True # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response['output_text']}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: 2000 Toyota Rav 4 Brakes


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


 **SBG**: For a 2000 Toyota RAV4, the brake pad replacement costs are as follows: 
- 3.2 Automatic SUV: $1.8
- 1.8 Automatic Pickup Trucks: $1.95
- 2.4 Automatic Sedan/Hatchback: $1.5

Additionally, the front brake reline cost is $1.2. 

There are also various disc brake options available, including:
- Pro-Series OE PEDIS4806: $201.95
- Pro-Series OE Plus PSDIS1684: $206.22
- Brembo BODIS6950: $183.41
- Centric CCDIS5254: $184.51

Please note that these prices may not include labor costs or other related expenses.


KeyboardInterrupt: Interrupted by user

## Using invoke method

In [ ]:
# Create a prompt template
chatbot_prompt = """
You are an automotive parts assistant. When a user asks about their vehicle, you will refer to relevant documents and provide guidance in a concise, clear manner. Your goal is to stay under 1,000 tokens for each response, including all necessary details.

Here are the steps to follow:

1. **Identify the Vehicle and Parts**: Determine what part the user is asking about based on their question. If it’s unclear, ask clarifying questions to understand the model, make, and year of the vehicle.

2. **Provide Pricing**: Always provide the price for the part requested. If you don’t have the exact year of the vehicle the user asks about, provide the price for the earliest year available in your database and inform the user that you cannot help with the exact year requested. You should always mention the earliest year available, even if the part is not available for the requested year.

3. **Include Related Subcategories**: If the part requested falls under a category that has subcategories (e.g., "engine parts" has "fuel injectors", "oil filters", etc.), list those subcategories with their prices, if available.

4. **Mention Fluids**: If the part requested is related to fluids (e.g., oil, transmission fluid), also mention the fluids associated with the part and their availability/price.

5. **Maintain Chat Context**: Keep track of previous conversations and refer to them when necessary to provide consistent follow-up answers. For example, if the user has already asked about a part and later asks about another part from the same vehicle, refer back to previous details such as model, year, or part-related info.

6. **Structure the Response**:
   - Begin by acknowledging the vehicle type and confirming details, especially if the model year or make was mentioned.
   - Provide the price for the part.
   - List any relevant subcategories and related fluids.
   - If no exact match for the year is found, state the earliest year available in the database and explain the limitation.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.invoke(
            {"input_documents": combined_results, "question": query}, return_only_outputs=True # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response['output_text']}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: 2001 Toyota Rav4 Brakes


BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

# Manual entry of vector database - Practice only

In [ ]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")


Extracting /content/vector_db_parts_2001.zip...
Extraction completed for vector_db_parts_2001.zip!
Extracting /content/vectordb_book1.zip...
Extraction completed for vectordb_book1.zip!


In [ ]:
# Retrieving the first vector database
persist_directory_1 = "/content/content/auto_book_1"
vectordb_1 = Chroma(
    persist_directory=persist_directory_1,
    embedding_function=embeddings
)

# Retrieving the second vector database
persist_directory_2 = "/content/content/chroma_auto_parts_2001"
vectordb_2 = Chroma(
    persist_directory=persist_directory_2,
    embedding_function=embeddings
)
"""
# Retrieving the third vector database
persist_directory_3 = "/content/content/auto_book_2"
vectordb_3 = Chroma(
    persist_directory=persist_directory_3,
    embedding_function=embeddings
)
"""
# Create retrievers for both vector databases
retriever_1 = vectordb_1.as_retriever()
retriever_2 = vectordb_2.as_retriever()
#retriever_3 = vectordb_3.as_retriever()

# Define a combined retriever function
def combined_retriever(query):
    # Retrieve documents from both retrievers
    results_1 = retriever_1.get_relevant_documents(query)
    results_2 = retriever_2.get_relevant_documents(query)
    #results_3 = retriever_3.get_relevant_documents(query)

    # Combine results (you can sort or filter if needed)
    combined_results = results_1 + results_2 #+ results_3
    return combined_results

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce") # "stuff" , "refine"


<ipython-input-3-97b26d130e9c>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb_1 = Chroma(


In [ ]:
# Create a prompt template
chatbot_prompt = """
You are an automotive parts assistant. When a user asks about their vehicle, you will refer to relevant documents and provide guidance in a concise, clear manner. Your goal is to stay under 1,000 tokens for each response, including all necessary details.

Here are the steps to follow:

1. **Identify the Vehicle and Parts**: Determine what part the user is asking about based on their question. If it’s unclear, ask clarifying questions to understand the model, make, and year of the vehicle.

2. **Provide Pricing**: Always provide the price for the part requested. If you don’t have the exact year of the vehicle the user asks about, provide the price for the earliest year available in your database and inform the user that you cannot help with the exact year requested. You should always mention the earliest year available, even if the part is not available for the requested year.

3. **Include Related Subcategories**: If the part requested falls under a category that has subcategories (e.g., "engine parts" has "fuel injectors", "oil filters", etc.), list those subcategories with their prices, if available.

4. **Mention Fluids**: If the part requested is related to fluids (e.g., oil, transmission fluid), also mention the fluids associated with the part and their availability/price.

5. **Maintain Chat Context**: Keep track of previous conversations and refer to them when necessary to provide consistent follow-up answers. For example, if the user has already asked about a part and later asks about another part from the same vehicle, refer back to previous details such as model, year, or part-related info.

6. **Structure the Response**:
   - Begin by acknowledging the vehicle type and confirming details, especially if the model year or make was mentioned.
   - Provide the price for the part.
   - List any relevant subcategories and related fluids.
   - If no exact match for the year is found, state the earliest year available in the database and explain the limitation.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}

"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.run(
            input_documents=combined_results,
            question=prompt  # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: My tire is falt


<ipython-input-3-97b26d130e9c>:30: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  results_1 = retriever_1.get_relevant_documents(query)
<ipython-input-4-1d25b260fe23>:62: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  response = qa_chain.run(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1554 > 1024). Running this sequence through the model will result in indexing errors



 **SBG**: I think you meant to say "My tire is flat." I'd be happy to help you with that. Can you please tell me what kind of vehicle you have, including the make, model, and year? This will help me provide more accurate information about your tire.

However, I can provide some general information about tires. According to our documentation, tire-related topics include:

- Tire pressure monitoring systems (TPMSs)
- Tire construction
- Load ratings
- DOT tire codes
- Inspecting and installing tires

Additionally, we have information on tire services such as:

* Tire Inflation
* Checking Air Pressure
* Adjusting Tire Pressure
* Tire Wear
* Sidewall Checks
* Tire Rotation
* Removing and Tightening Lug Nuts
* Removing and Mounting Tires on Rims
* Inspecting the Tire and Wheel
* Valve Stem Service
* Tire Repair

Please provide more details about your vehicle so I can offer more specific guidance and pricing for the required services.

 **You**: 2001 Toyota Corolla brakes

 **SBG**: For you